In [1746]:
import pandas as pd
import numpy as np

In [1747]:
df=pd.read_csv('lung_cancer_dataset\cancer patient data sets.csv')
# 2. Encode the Level column FIRST
mapping = {'High': 2, 'Medium': 1, 'Low': 0}
df['Level'] = df['Level'].map(mapping)
# Drop the index and Patient Id columns (they're not real features)
df = df.drop(['index', 'Patient Id'], axis=1)


X = df.drop('Level', axis=1).values
y = df['Level'].values

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Rujjul Saha\AppData\Local\Temp\ipykernel_5424\2152436614.py:1: SyntaxWarning: invalid escape sequence '\c'
  df=pd.read_csv('lung_cancer_dataset\cancer patient data sets.csv')


In [1748]:
# TECHNIQUE 1: INTRODUCE CLASS LABEL NOISE (Flip some labels)

def add_label_noise(y, noise_rate=0.05):
    """Randomly flip some labels to adjacent classes"""
    y_noisy = y.copy()
    n_samples = len(y)
    n_noisy = int(n_samples * noise_rate)
    
    # Randomly select samples to flip
    noisy_indices = np.random.choice(n_samples, n_noisy, replace=False)
    
    for idx in noisy_indices:
        current_label = y_noisy[idx]
        # Flip to adjacent class
        if current_label == 0:  # Low
            y_noisy[idx] = 1  # To Medium
        elif current_label == 1:  # Medium
            y_noisy[idx] = np.random.choice([0, 2])  # To Low or High
        else:  # High
            y_noisy[idx] = 1  # To Medium
    
    return y_noisy


In [1749]:
# TECHNIQUE 2: ADD IRRELEVANT/NOISY FEATURES

def add_noisy_features(df, n_features=5):
    """Add random irrelevant features"""
    df_new = df.copy()
    
    for i in range(n_features):
        # Random noise features
        df_new[f'noise_feature_{i+1}'] = np.random.normal(5, 2, len(df_new))
    
    return df_new



complexity_levels = {
    'Original': df.copy(),
    'Label Noise (10%)': df.copy(),  # Will modify labels during split
    'Added Noisy Features': add_noisy_features(df, n_features=10),  
}

results = []

In [1750]:
for complexity_name, df_complex in complexity_levels.items():
    # Add label noise if specified
    if complexity_name == 'Label Noise (10%)':
        y = add_label_noise(y, noise_rate=0.10)
        print(f"⚠️  10% of labels randomly flipped")

⚠️  10% of labels randomly flipped


In [1751]:
df.head()

,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
0,33,1,2,4,5,4,3,2,2,4,...,3,4,2,2,3,1,2,3,4,0
1,17,1,3,1,5,3,4,2,2,2,...,1,3,7,8,6,2,1,7,2,1
2,35,1,4,5,6,5,5,4,6,7,...,8,7,9,2,1,4,6,7,2,2
3,37,1,7,7,7,7,6,7,7,7,...,4,2,3,1,4,5,6,7,5,2
4,46,1,6,8,7,7,7,6,7,7,...,3,2,4,1,4,2,4,2,3,2


In [1752]:
df.shape

(1000, 24)

In [1753]:
df.dtypes

Age                         int64
Gender                      int64
Air Pollution               int64
Alcohol use                 int64
Dust Allergy                int64
OccuPational Hazards        int64
Genetic Risk                int64
chronic Lung Disease        int64
Balanced Diet               int64
Obesity                     int64
Smoking                     int64
Passive Smoker              int64
Chest Pain                  int64
Coughing of Blood           int64
Fatigue                     int64
Weight Loss                 int64
Shortness of Breath         int64
Wheezing                    int64
Swallowing Difficulty       int64
Clubbing of Finger Nails    int64
Frequent Cold               int64
Dry Cough                   int64
Snoring                     int64
Level                       int64
dtype: object

In [1754]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("y unique values:", np.unique(y))

X shape: (1000, 23)
y shape: (1000,)
y unique values: [0 1 2]


In [1755]:
column_names_for_X = df.columns[2:-1].tolist()
print("The column names for X are:")
print(column_names_for_X)

The column names for X are:
['Air Pollution', 'Alcohol use', 'Dust Allergy', 'OccuPational Hazards', 'Genetic Risk', 'chronic Lung Disease', 'Balanced Diet', 'Obesity', 'Smoking', 'Passive Smoker', 'Chest Pain', 'Coughing of Blood', 'Fatigue', 'Weight Loss', 'Shortness of Breath', 'Wheezing', 'Swallowing Difficulty', 'Clubbing of Finger Nails', 'Frequent Cold', 'Dry Cough', 'Snoring']


In [1756]:
column_names_for_y = df.columns[[-1]].tolist()
print("The column names for y are:")
print(column_names_for_y)

The column names for y are:
['Level']


In [1757]:
# Check each feature individually
print("\n" + "="*80)
print("CHECKING EACH FEATURE FOR PERFECT SEPARATION")
print("="*80)

features = df.drop('Level', axis=1).columns

for feature in features:
    # Check if this single feature can perfectly predict Level
    unique_combinations = df.groupby(feature)['Level'].nunique()
    
    # If each unique value of the feature maps to only one Level
    if (unique_combinations == 1).all():
        print(f"\n🚨 PERFECT PREDICTOR FOUND: {feature}")
        print(f"Each unique value of '{feature}' maps to exactly one Level:")
        print(df.groupby(feature)['Level'].unique())
    
# Also check combinations of features
print("\n" + "="*80)
print("DETAILED CORRELATION ANALYSIS")
print("="*80)

correlations = df.corr()['Level'].abs().sort_values(ascending=False)
print(correlations)

# Check if any single feature has 100% accuracy
print("\n" + "="*80)
print("TESTING SINGLE FEATURES FOR PERFECT PREDICTION")
print("="*80)


CHECKING EACH FEATURE FOR PERFECT SEPARATION

DETAILED CORRELATION ANALYSIS
Level                       1.000000
Obesity                     0.827435
Coughing of Blood           0.782092
Alcohol use                 0.718710
Dust Allergy                0.713839
Balanced Diet               0.706273
Passive Smoker              0.703594
Genetic Risk                0.701303
OccuPational Hazards        0.673255
Chest Pain                  0.645461
Air Pollution               0.636038
Fatigue                     0.625114
chronic Lung Disease        0.609971
Smoking                     0.519530
Shortness of Breath         0.497024
Frequent Cold               0.444017
Dry Cough                   0.373968
Weight Loss                 0.352738
Snoring                     0.289366
Clubbing of Finger Nails    0.280063
Swallowing Difficulty       0.249142
Wheezing                    0.242794
Gender                      0.164985
Age                         0.060048
Name: Level, dtype: float64

TESTIN

## Splitting the dataset into the Training set and Test set

In [1758]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [1759]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [1760]:
from collections import Counter# Check counts 
print(Counter(y_train))


Counter({1: 278, 2: 258, 0: 214})


## Training the Logistic Regression model on the Training set

In [1761]:
from sklearn.linear_model import LogisticRegression
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [1762]:
y_pred_lr = classifier_lr.predict(X_test)

In [1763]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
#Confusion Matrix
print("Confusion Matrix for Logistic Regression:")
cm = confusion_matrix(y_test, y_pred_lr)
print(cm)
# Accuracy
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("\nAccuracy (Logistic Regression):", accuracy_lr)
#Recall
recall_lr = recall_score(y_test, y_pred_lr, average='weighted')
print("\nRecall (Logistic Regression):", recall_lr)
#Precision
precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
print("\nPrecision (Logistic Regression):",precision_lr)
#F1 Score
f1_lr = f1_score(y_test,y_pred_lr, average='weighted')
print("\nF1 Score (Logistic Regression):",f1_lr)
#Classification Report
report_lr = classification_report(y_test, y_pred_lr)
print("\nClassification Report(Logistic Regression):\n",report_lr)

Confusion Matrix for Logistic Regression:
[[76  3  0]
 [ 9 67  8]
 [ 0  4 83]]

Accuracy (Logistic Regression): 0.904

Recall (Logistic Regression): 0.904

Precision (Logistic Regression): 0.9041639860933979

F1 Score (Logistic Regression): 0.90237939993964

Classification Report(Logistic Regression):
               precision    recall  f1-score   support

           0       0.89      0.96      0.93        79
           1       0.91      0.80      0.85        84
           2       0.91      0.95      0.93        87

    accuracy                           0.90       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.90      0.90      0.90       250



In [1764]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier_lr, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 89.87 %
Standard Deviation: 1.15 %


## Training the KNN model on the Training set


In [1765]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier()

In [1766]:
y_pred_knn = classifier_knn.predict(X_test)

In [1767]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
#Confusion Matrix
print("Confusion Matrix for KNN:")
cm = confusion_matrix(y_test, y_pred_knn)
print(cm)
# Accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("\nAccuracy (KNN):", accuracy_knn)
#Recall
recall_knn = recall_score(y_test, y_pred_knn, average='weighted')
print("\nRecall (KNN):", recall_knn)
#Precision
precision_knn = precision_score(y_test, y_pred_knn, average='weighted')
print("\nPrecision (KNN):",precision_knn)
#F1 Score
f1_knn = f1_score(y_test,y_pred_knn, average='weighted')
print("\nF1 Score (KNN):",f1_knn)
#Classification Report
report_knn = classification_report(y_test, y_pred_knn)
print("\nClassification Report(KNN):\n",report_knn)

Confusion Matrix for KNN:
[[72  7  0]
 [ 9 68  7]
 [ 0  5 82]]

Accuracy (KNN): 0.888

Recall (KNN): 0.888

Precision (KNN): 0.8871181023720348

F1 Score (KNN): 0.8873068736141906

Classification Report(KNN):
               precision    recall  f1-score   support

           0       0.89      0.91      0.90        79
           1       0.85      0.81      0.83        84
           2       0.92      0.94      0.93        87

    accuracy                           0.89       250
   macro avg       0.89      0.89      0.89       250
weighted avg       0.89      0.89      0.89       250



In [1768]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier_knn, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 88.93 %
Standard Deviation: 1.77 %


In [1769]:
## Training the Supper Vector Machine model on the Training set
from sklearn.svm import SVC
classifier_svm = SVC(kernel = 'linear',class_weight='balanced', random_state = 0)
classifier_svm.fit(X_train, y_train)


SVC(class_weight='balanced', kernel='linear', random_state=0)

In [1770]:
y_pred_svm = classifier_svm.predict(X_test)


In [1771]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
#Confusion Matrix
print("Confusion Matrix for SVM:")
cm = confusion_matrix(y_test, y_pred_svm)
print(cm)
# Accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("\nAccuracy (SVM):", accuracy_svm)
#Recall
recall_svm = recall_score(y_test, y_pred_svm, average='weighted')
print("\nRecall (SVM):", recall_svm)
#Precision
precision_svm = precision_score(y_test, y_pred_svm, average='weighted')
print("\nPrecision (SVM):",precision_svm)
#F1 Score
f1_svm = f1_score(y_test,y_pred_svm, average='weighted')
print("\nF1 Score (SVM):",f1_svm)
#Classification Report
report_svm = classification_report(y_test, y_pred_svm)
print("\nClassification Report(SVM):\n",report_svm)


Confusion Matrix for SVM:
[[76  3  0]
 [ 9 67  8]
 [ 0  4 83]]

Accuracy (SVM): 0.904

Recall (SVM): 0.904

Precision (SVM): 0.9041639860933979

F1 Score (SVM): 0.90237939993964

Classification Report(SVM):
               precision    recall  f1-score   support

           0       0.89      0.96      0.93        79
           1       0.91      0.80      0.85        84
           2       0.91      0.95      0.93        87

    accuracy                           0.90       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.90      0.90      0.90       250



In [1772]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier_svm, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))


Accuracy: 89.87 %
Standard Deviation: 1.15 %


## Training the Random Forest Classifier model on the Training set


In [1773]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(n_estimators = 100,max_depth=5,min_samples_leaf=5, criterion = 'entropy',class_weight='balanced', random_state = 0)
classifier_rf.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=5, min_samples_leaf=5, random_state=0)

In [1774]:
y_pred_rf = classifier_rf.predict(X_test)


In [1775]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
#Confusion Matrix
print("Confusion Matrix for Random Forest:")
cm = confusion_matrix(y_test, y_pred_rf)
print(cm)
# Accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("\nAccuracy (Random Forest):", accuracy_rf)
#Recall
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
print("\nRecall (Random Forest):", recall_rf)
#Precision
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
print("\nPrecision (Random Forest):",precision_rf)
#F1 Score
f1_rf = f1_score(y_test,y_pred_rf, average='weighted')
print("\nF1 Score (Random Forest):",f1_rf)
#Classification Report
report_rf = classification_report(y_test, y_pred_rf)
print("\nClassification Report(Random Forest):\n",report_rf)


Confusion Matrix for Random Forest:
[[76  3  0]
 [ 9 67  8]
 [ 0  4 83]]

Accuracy (Random Forest): 0.904

Recall (Random Forest): 0.904

Precision (Random Forest): 0.9041639860933979

F1 Score (Random Forest): 0.90237939993964

Classification Report(Random Forest):
               precision    recall  f1-score   support

           0       0.89      0.96      0.93        79
           1       0.91      0.80      0.85        84
           2       0.91      0.95      0.93        87

    accuracy                           0.90       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.90      0.90      0.90       250



In [1776]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier_rf, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))


Accuracy: 89.47 %
Standard Deviation: 1.54 %


## Training the Decision Tree Classifier model on the Training set


In [1777]:
from sklearn.tree import DecisionTreeClassifier
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)


DecisionTreeClassifier(criterion='entropy', random_state=0)

In [1778]:
y_pred_dt = classifier_dt.predict(X_test)


In [1779]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
#Confusion Matrix
print("Confusion Matrix for Decision Tree:")
cm = confusion_matrix(y_test, y_pred_dt)
print(cm)
# Accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("\nAccuracy (Decision Tree):", accuracy_dt)
#Recall
recall_dt = recall_score(y_test, y_pred_dt, average='weighted')
print("\nRecall (Decision Tree):", recall_dt)
#Precision
precision_dt = precision_score(y_test, y_pred_dt, average='weighted')
print("\nPrecision (Decision Tree):",precision_dt)
#F1 Score
f1_dt = f1_score(y_test,y_pred_dt, average='weighted')
print("\nF1 Score (Decision Tree):",f1_dt)
#Classification Report
report_dt = classification_report(y_test, y_pred_dt)
print("\nClassification Report(Decision Tree):\n",report_dt)


Confusion Matrix for Decision Tree:
[[72  7  0]
 [ 9 67  8]
 [ 0 10 77]]

Accuracy (Decision Tree): 0.864

Recall (Decision Tree): 0.864

Precision (Decision Tree): 0.8641359477124183

F1 Score (Decision Tree): 0.8639813953488374

Classification Report(Decision Tree):
               precision    recall  f1-score   support

           0       0.89      0.91      0.90        79
           1       0.80      0.80      0.80        84
           2       0.91      0.89      0.90        87

    accuracy                           0.86       250
   macro avg       0.86      0.86      0.86       250
weighted avg       0.86      0.86      0.86       250



In [1780]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier_dt, X = X_train, y = y_train, cv = 5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))


Accuracy: 88.93 %
Standard Deviation: 1.61 %


## Training the XG Boost Classifier model on the Training set


In [1781]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

classifier_xgb = XGBClassifier(
    n_estimators=100,
    learning_rate=0.05,    # Slow learning for stability
    max_depth=3,           # Shallow trees to prevent overfitting
    subsample=0.8,         # Use random subset of rows
    colsample_bytree=0.8,  # Use random subset of columns
    eval_metric='logloss',
    random_state=0
)
classifier_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [1782]:
y_pred_xgb = classifier_xgb.predict(X_test)


In [1783]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
#Confusion Matrix
print("Confusion Matrix for XGB:")
cm = confusion_matrix(y_test, y_pred_xgb)
print(cm)
# Accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("\nAccuracy (XGB):", accuracy_xgb)
#Recall
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')
print("\nRecall (XGB):", recall_xgb)
#Precision
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
print("\nPrecision (XGB):",precision_xgb)
#F1 Score
f1_xgb = f1_score(y_test,y_pred_xgb, average='weighted')
print("\nF1 Score (XGB):",f1_xgb)
#Classification Report
report_xgb = classification_report(y_test, y_pred_xgb)
print("\nClassification Report(XGB):\n",report_xgb)

Confusion Matrix for XGB:
[[76  3  0]
 [ 9 67  8]
 [ 0  4 83]]

Accuracy (XGB): 0.904

Recall (XGB): 0.904

Precision (XGB): 0.9041639860933979

F1 Score (XGB): 0.90237939993964

Classification Report(XGB):
               precision    recall  f1-score   support

           0       0.89      0.96      0.93        79
           1       0.91      0.80      0.85        84
           2       0.91      0.95      0.93        87

    accuracy                           0.90       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.90      0.90      0.90       250



In [1784]:

accuracies = cross_val_score(estimator=classifier_xgb, X=X_train, y=y_train, cv=5)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))


Accuracy: 89.73 %
Standard Deviation: 1.24 %
